In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [2]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.exceptions import NotFittedError
from skforecast.preprocessing import RollingFeatures
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor

In [29]:
# Create and fit forecaster
# ==============================================================================
rolling = RollingFeatures(stats= ['mean', 'median', 'sum'], window_sizes=[5, 4, 5])
forecaster = ForecasterAutoregDirect(
                 regressor       = LinearRegression(),
                 steps           = 2,
                 lags            = None,
                 window_features= rolling,
                 differentiation = 1
             )

y = pd.Series(np.arange(10), name='y')
exog = pd.Series(np.arange(100, 110), name='exog')
exog_predict = pd.Series(np.arange(110, 112), name='exog')
exog_predict.index = pd.RangeIndex(start=10, stop=12, step=1)
forecaster.fit(y=y, exog=exog)

In [30]:
forecaster._create_train_X_y(y=y, exog=exog)

(   roll_mean_5  roll_median_4  roll_sum_5  exog_step_1  exog_step_2
 7          1.0            1.0         5.0        106.0        107.0
 8          1.0            1.0         5.0        107.0        108.0
 9          1.0            1.0         5.0        108.0        109.0,
 {1: 6    1.0
  7    1.0
  8    1.0
  Name: y_step_1, dtype: float64,
  2: 7    1.0
  8    1.0
  9    1.0
  Name: y_step_2, dtype: float64},
 ['exog'],
 ['roll_mean_5', 'roll_median_4', 'roll_sum_5'],
 ['exog'],
 ['roll_mean_5', 'roll_median_4', 'roll_sum_5', 'exog_step_1', 'exog_step_2'],
 {'exog': dtype('int64')})

In [93]:
forecaster.create_predict_X(exog=exog_predict)

['roll_mean_5', 'roll_median_4', 'roll_sum_5', 'exog']


,roll_mean_5,roll_median_4,roll_sum_5,exog
10,7.0,7.5,35.0,110.0
11,7.0,7.5,35.0,111.0


In [95]:
forecaster.predict(exog=exog_predict)

10    10.0
11    11.0
Name: pred, dtype: float64

In [35]:
rolling.transform_batch(y)

,roll_mean_4,roll_median_4,roll_sum_5
5,2.5,2.5,10.0
6,3.5,3.5,15.0
7,4.5,4.5,20.0
8,5.5,5.5,25.0
9,6.5,6.5,30.0


In [96]:
y = pd.Series(np.arange(5), name='y')
forecaster = ForecasterAutoregDirect(LinearRegression(), lags=3, steps=2)

forecaster.create_train_X_y(y=y)

(   lag_1  lag_2  lag_3
 4    2.0    1.0    0.0,
 {1: 3    3.0
  Name: y_step_1, dtype: float64,
  2: 4    4.0
  Name: y_step_2, dtype: float64})

In [6]:
from skforecast.preprocessing import RollingFeatures
from skforecast.preprocessing import TimeSeriesDifferentiator
from skforecast.ForecasterAutoreg import ForecasterAutoreg

# Fixtures
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import y
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import exog
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import exog_predict
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import data  # to test results when using differentiation


In [19]:
y_datetime = y.copy()
y_datetime.index = pd.date_range(start='2001-01-01', periods=len(y), freq='D')
exog_datetime = exog.copy()
exog_datetime.index = pd.date_range(start='2001-01-01', periods=len(exog), freq='D')
exog_predict_datetime = exog_predict.copy()
exog_predict_datetime.index = pd.date_range(start='2001-02-20', periods=len(exog_predict), freq='D')
rolling = RollingFeatures(stats=['mean', 'sum'], window_sizes=3)
forecaster = ForecasterAutoreg(
    LGBMRegressor(verbose=-1, random_state=123), lags=3, window_features=rolling,
    differentiation=1
)
forecaster.fit(y=y_datetime, exog=exog_datetime)

In [ ]:

results = forecaster.predict_bootstrapping(
    steps=4, n_boot=10, exog=exog_predict_datetime, use_in_sample_residuals=True
)
results

In [5]:
results.to_numpy()

array([[0.61866004, 0.89697256, 0.14325579, 0.18128852, 0.86846565,
        0.55020849, 0.25870679, 0.51670634, 0.29239436, 0.42315364],
       [0.26475697, 0.67230222, 0.18926364, 0.45124373, 0.34317802,
        0.6900186 , 0.43134277, 0.67230222, 0.20674374, 0.84082826],
       [0.40053981, 0.32295891, 0.61577585, 0.15682665, 0.43966009,
        0.46585818, 0.5110144 , 0.5225711 , 0.56031677, 0.47247859],
       [0.39037942, 0.06805095, 0.86630916, 0.52190308, 0.47247859,
        0.43966009, 0.46227478, 0.58525714, 0.5189877 , 0.28131971]])